In [1]:
# coding: utf-8
import numpy as np
import pandas as pd
import time
from configparser import ConfigParser, ExtendedInterpolation

In [2]:
config = ConfigParser(inline_comment_prefixes="#;", interpolation=ExtendedInterpolation())
config.read('config.ini')
xlsfile = config['General']['output_file'] # output of generate trends is input here

In [3]:
big_topics = pd.read_excel(xlsfile, sheetname=1).iloc[:,-1]
big_topics = (big_topics[~big_topics.isnull()])[:100].to_list()

/usr/local/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [4]:
def readData(filename):
    print('Reading data....')
    start = time.time()
    df = pd.read_pickle(filename)
    end = time.time()
    print(f'Read finished in {end-start:.2f} seconds.\n')
    return df

In [5]:
inputfile = config['Text Cleaning']['tokenized_file']
df = readData(inputfile)

Reading data....
Read finished in 3.17 seconds.



In [6]:
data = df.reset_index()

In [7]:
start_date = '01-01-2010'
end_date = '30-06-2019'
data['Date'] = pd.to_datetime(data['Date'])

In [10]:
mask = (data['Date']>=start_date) & (data['Date']<=end_date)
data = data[mask].Bigrams

In [11]:
idxs = []
for idx, row in enumerate(data.to_list()):
    for topic in row:
        if topic in big_topics:
            idxs.append(idx)

In [16]:
texts = df.ix[idxs].reset_index().Bigrams
texts.shape

/usr/local/Cellar/jupyter/1.0.0_5/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


(3845,)

In [17]:
from gensim.models import LdaModel, LsiModel, HdpModel
from gensim.corpora import Dictionary
from gensim.models.wrappers import LdaMallet
import pyLDAvis.gensim

In [18]:
dictionary = Dictionary(texts)

In [19]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
#use lda multicore

In [20]:
ldamodel = LdaModel(corpus=corpus,
                    num_topics=12, 
                    id2word=dictionary, 
                    random_state=42, 
                    update_every=1, 
                    chunksize=100, 
                    passes=10, 
                    alpha='auto', 
                    per_word_topics=True)

In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()

In [23]:
start = time.time()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
end = time.time()
print(f'Prepared visualization in {end-start:.2f} seconds.\n')

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Prepared visualization in 204.77 seconds.



In [24]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.063080  0.008926       1        1  17.982716
1      0.210660  0.334783       2        1  15.131645
8     -0.111628 -0.022134       3        1  15.016395
2      0.138998  0.069553       4        1  13.280995
9     -0.069609 -0.023127       5        1  10.688872
0      0.357984 -0.264987       6        1   9.743878
11    -0.047486 -0.019206       7        1   6.959298
3     -0.112897 -0.021435       8        1   6.175349
5     -0.108745 -0.016347       9        1   3.646617
4     -0.106785 -0.015342      10        1   0.460414
6     -0.106785 -0.015342      11        1   0.456975
10    -0.106785 -0.015342      12        1   0.456852, topic_info=     Category        Freq                    Term       Total  loglift  \
1127  Default  568.000000          climate change  568.000000  30.0000   
1716  Default  387.000000         dangers climate  387.000000  29.0000   
1387  Default  386.000000          mass migration  386.000000  28.0000   
4917  Default  386.000000             one dangers  386.000000  27.0000   
4916  Default  386.000000           migration big  386.000000  26.0000   
4915  Default  386.000000             change mass  386.000000  25.0000   
11    Default  396.000000             people know  396.000000  24.0000   
465   Default  370.000000             kill people  370.000000  23.0000   
1056  Default  369.000000           designed kill  369.000000  22.0000   
510   Default  369.000000        malware designed  369.000000  21.0000   
4116  Default  369.000000         crafted malware  369.000000  20.0000   
4117  Default  369.000000         hackers crafted  369.000000  19.0000   
191   Default  299.000000             made simple  299.000000  18.0000   
990   Default  298.000000        researchers made  298.000000  17.0000   
4873  Default  297.000000      dramatic discovery  297.000000  16.0000   
4875  Default  297.000000         simple dramatic  297.000000  15.0000   
4869  Default  288.000000         training neural  288.000000  14.0000   
4867  Default  288.000000          networks wrong  288.000000  13.0000   
4868  Default  288.000000         neural networks  288.000000  12.0000   
4883  Default  287.000000      discovery training  287.000000  11.0000   
731   Default  240.000000             lung cancer  240.000000  10.0000   
2164  Default  239.000000           learning soon  239.000000   9.0000   
2165  Default  239.000000        machine learning  239.000000   8.0000   
3175  Default  239.000000             detect lung  239.000000   7.0000   
1396  Default  239.000000             soon detect  239.000000   6.0000   
2490  Default  239.000000         algorithm shows  239.000000   5.0000   
2108  Default  239.000000           shows machine  239.000000   4.0000   
117   Default  239.000000           cancer faster  239.000000   3.0000   
4936  Default  239.000000         faster reliably  239.000000   2.0000   
131   Default  202.000000               create 3d  202.000000   1.0000   
...       ...         ...                     ...         ...      ...   
1819  Topic12    0.020867          model recreate    1.316656   1.2439   
1818  Topic12    0.020867              learn idea    1.316656   1.2439   
5279  Topic12    0.020867        technology sweat    2.356434   0.6618   
1753  Topic12    0.020867             hole ejects    1.318439   1.2425   
1755  Topic12    0.020867          massive energy    1.318439   1.2425   
1756  Topic12    0.020867           burped plasma    1.318387   1.2426   
1757  Topic12    0.020867         check extremely    1.318387   1.2426   
1758  Topic12    0.020867             eating star    1.318387   1.2426   
1759  Topic12    0.020867          extremely rare    1.318387   1.2426   
1760  Topic12    0.020867             hole burped    1.318387   1.2426   
1761  Topic12    0.020867           plasma eating    1.318387   1.2426   
1762  Topic12    0.

In [ ]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)

In [ ]:
hdpmodel.show_topics()

In [ ]:
import os
os.environ.update({'MALLET_HOME':"/usr/local/opt/mallet-2.0.8/bin/mallet"})# update this path accordingly

mallet_path = "/usr/local/opt/mallet-2.0.8/bin/mallet" # update this path accordingly

In [ ]:
ldamallet = LdaMallet(mallet_path, 
                        corpus=corpus, 
                        num_topics=12, 
                        id2word=dictionary,   
                        workers=4)